## Cryptocurrency trading bot.

### Install if needed.

In [ ]:
##%pip install python-binance==0.7.5
#%pip install --upgrade python-binance
#%pip install paramiko

### Prelude.

In [ ]:
# Import libraries.
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table

# Manage API keys.
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client

# Get all available pair information for trading.
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

# Precalculate UTC offset for inter-server communication coherence.
offset_s = get_timezone_offset_in_seconds()

# Precalculate conversion_table.
conversion_table = get_conversion_table(client=client, 
                                        exchange_info=exchange_info, 
                                        offset_s=offset_s, 
                                        dump_raw=False, 
                                        as_pair=True, 
                                        minimal=False, 
                                        extra_minimal=False, 
                                        super_extra_minimal=False, 
                                        convert_to_USDT=False)

# Display pre-computed conversion table.
conversion_table

### This is how to synchronize the wallet with the virtual wallet (take all above zero).

In [ ]:
from cryptocurrency.trader.wallet import select_asset_with_biggest_wallet
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, 
                                     conversion_table=conversion_table, 
                                     exchange_info=exchange_info)
priority = 'fees' if float(converted_quantity) > 10.0 else 'wallet'
from_asset, converted_quantity, quantity

### Trade.

In [ ]:
to_asset = 'BTC'

from cryptocurrency.trader.trade import trade
import pandas as pd

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                exchange_info=exchange_info, priority=priority)
from_asset = to_asset
request

### Main loop buying screened assets in pairs which are connected to the wallet holdings.

In [ ]:
sell_asset = 'BUSD'
take_profit = None
stop_loss = None
purchased_asset = None
asset_to_delete = None
keys_file = 'server_keys.txt'
input_log = '~/workspace/crypto_logs/crypto_input_log_15s.txt'
output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import select_pair_with_highest_quote_volume_from_base_asset
from cryptocurrency.trader.ssh import Ssh
from cryptocurrency.trader.wallet import select_asset_with_biggest_wallet
from cryptocurrency.trader.order_book import get_order_book_trigger
from cryptocurrency.trader.trade import trade
import time

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client

ssh = Ssh(input_log=input_log, output_log_screened=output_log_screened, keys_file=keys_file)

def choose_to_asset(from_asset, to_asset, latest_asset, output_log_screened=output_log_screened):
    tradable_pairs = ssh.get_logs_from_server(server_log=ssh.output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        if from_asset in tradable_assets:
            to_asset = latest_asset = from_asset
        else:
            buyable = False
            for test_asset in tradable_assets[::-1]:
                test_symbol = \
                    select_pair_with_highest_quote_volume_from_base_asset(base_asset=test_asset, 
                                                                          conversion_table=conversion_table, 
                                                                          exchange_info=exchange_info)
                time.sleep(1.0)
                if get_order_book_trigger(client=client, symbol=test_symbol, threshold=10000):
                    latest_asset = test_asset
                    buyable = True
                    break
            to_asset = latest_asset if buyable else sell_asset
    return latest_asset, to_asset

def trade_conditionally(client, exchange_info, to_asset, price_when_bought, asset_to_delete):
    #conversion_table = ssh.get_logs_from_server(server_log=ssh.input_log)
    conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
    from_asset, converted_quantity, quantity = \
        select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                         exchange_info=exchange_info)
    priority = 'fees' if float(converted_quantity) > 10.0 else 'wallet'
    request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                    exchange_info=exchange_info, priority=priority)
    if request is not None:
        if to_asset != sell_asset:
            pair_bought = request['symbol']
            price_when_bought = float(request['fills'][0]['price'])
        else:
            asset_to_delete = from_asset
        from_asset = to_asset
    return from_asset, to_asset, price_when_bought, asset_to_delete

latest_asset = from_asset
price_when_bought = quantity
while True:
    latest_asset, to_asset = choose_to_asset(from_asset, to_asset, latest_asset, 
                                             output_log_screened=output_log_screened)
    if from_asset != to_asset:
        from_asset, to_asset, price_when_bought, asset_to_delete = \
            trade_conditionally(client=client, exchange_info=exchange_info, to_asset=to_asset, 
                                price_when_bought=price_when_bought, asset_to_delete=asset_to_delete)
    time.sleep(2)

ssh.close()

### Rough draft from the older Take Profits/Stop Loss implementation of the main above.

In [ ]:
sell_asset = 'BUSD'
take_profit = 10.0
stop_loss = None
purchased_asset = None
asset_to_delete = None
keys_file = 'server_keys.txt'
input_log = '~/workspace/crypto_logs/crypto_input_log_15s.txt'
output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import select_pair_with_highest_quote_volume_from_base_asset
from cryptocurrency.trader.ssh import Ssh
from cryptocurrency.trader.wallet import select_asset_with_biggest_wallet
from cryptocurrency.trader.order_book import get_order_book_trigger
from cryptocurrency.trader.trade import trade
import time
import pandas as pd

ssh = Ssh(input_log=input_log, output_log_screened=output_log_screened, keys_file=keys_file)

def choose_to_asset(from_asset, to_asset, latest_asset, output_log_screened=output_log_screened):
    tradable_pairs = ssh.get_logs_from_server(server_log=output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        if from_asset in tradable_assets:
            to_asset = latest_asset = from_asset
        else:
            buyable = False
            for test_asset in tradable_assets[::-1]:
                test_symbol = \
                    select_pair_with_highest_quote_volume_from_base_asset(base_asset=test_asset, 
                                                                          conversion_table=conversion_table, 
                                                                          exchange_info=exchange_info)
                time.sleep(0.5)
                if get_order_book_trigger(client=client, symbol=test_symbol, threshold=10000):
                    latest_asset = test_asset
                    buyable = True
                    break
            to_asset = latest_asset if buyable else sell_asset
    return latest_asset, to_asset

def trade_conditionally(client, exchange_info, to_asset, price_when_bought, asset_to_delete):
    #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
    conversion_table = ssh.get_logs_from_server(server_log=ssh.input_log)
    from_asset, converted_quantity, quantity = \
        select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                         exchange_info=exchange_info)
    priority = 'fees' if float(converted_quantity) > 10.0 else 'wallet'
    request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                    exchange_info=exchange_info, priority=priority)
    if request is not None:
        if to_asset != sell_asset:
            pair_bought = request['symbol']
            price_when_bought = float(request['fills'][0]['price'])
        else:
            asset_to_delete = from_asset
        from_asset = to_asset
    return from_asset, to_asset, price_when_bought, asset_to_delete

latest_asset = from_asset
price_when_bought = quantity

while True:
    try:
        if asset_to_delete is None:
            latest_asset, to_asset = choose_to_asset(from_asset, to_asset, latest_asset, 
                                                     output_log_screened=output_log_screened)
            if from_asset != to_asset:
                from_asset, to_asset, price_when_bought, asset_to_delete = \
                    trade_conditionally(client=client, exchange_info=exchange_info, to_asset=to_asset, 
                                        price_when_bought=price_when_bought, asset_to_delete=asset_to_delete)
            elif purchased_asset is not None:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = ssh.get_logs_from_server(server_log=ssh.input_log)
                price_now = conversion_table[conversion_table['symbol'] == purchased_asset]['close'].iat[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            time.sleep(2)
        else:
            connected_pairs = exchange_info[exchange_info['base_asset'] == asset_to_delete]['symbol']
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[~(input_pairs['symbol'].isin(connected_pairs))]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()

### Example minimal loop using Paramiko.

In [ ]:
keys_file = 'server_keys.txt'
input_log = '~/workspace/crypto_logs/crypto_input_log_15s.txt'
output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import select_pair_with_highest_quote_volume_from_base_asset
from cryptocurrency.trader.ssh import Ssh
from cryptocurrency.trader.wallet import select_asset_with_biggest_wallet
from cryptocurrency.trader.order_book import get_order_book_trigger
from cryptocurrency.trader.trade import trade
import time

ssh = Ssh(input_log=input_log, output_log_screened=output_log_screened, keys_file=keys_file)

while True:
    #df = ssh.get_logs_from_server(server_log=ssh.input_log)
    df = ssh.get_logs_from_server(server_log=ssh.output_log_screened)

    if df.shape[0] < 1:
        print('None')
    else:
        print(df)

    time.sleep(2)

ssh.close()